In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='sgsapp'
mark='video'

In [4]:
# 载入模型
model_hour = '2020072019'
valid_hour = '2020072022'
# valid_hour = '2020030812'
#model_root = f'/search/odin/publicData/CloudS/libowei/rank_online/data/{mark}_hour_sgsapp_v1/models/8'
model_root = f'/search/odin/publicData/CloudS/mkyuwen/rank_online/data/video_hour_sgsapp_v1/models/1'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
# tfrecord_root = f'/search/odin/publicData/CloudS/libowei/rank/{product}/data/{mark}_hour_{product}_v1/tfrecords'
tfrecord_root = f'/search/odin/publicData/CloudS/yuwenmengke/rank_0521/sgsapp/data/video_hour_sgsapp_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
print('model:', model_dir)
print('record:', tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
# FLAGS.fields_norm=True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

model: /search/odin/publicData/CloudS/mkyuwen/rank_online/data/video_hour_sgsapp_v1/models/1/2020072019
record: /search/odin/publicData/CloudS/yuwenmengke/rank_0521/sgsapp/data/video_hour_sgsapp_v1/tfrecords/2020072022


2020-07-21 17:16:40 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-07-21 17:16:40 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-07-21 17:16:40 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-07-21 17:16:40 0:00:00 model: [1] model_dir: [/search/odin/mkyuwen/rank_onexp_0612/video/model/1]
2020-07-21 17:16:41 0:00:00 Latest checkpoint: /search/odin/publicData/CloudS/mkyuwen/rank_online/data/video_hour_sgsapp_v1/models/1/2020072019/model.ckpt-2020072019-1.00-3590


In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

1102904

In [21]:
batch_size = 512
batch_size = 1
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [22]:
def get_info(x, mid):
  mids = gezi.decode(x['mid'].numpy())
  hits = []
  for i in range(len(mids)):
    if mids[i] == mid:
      hits += [i]
  res = [{}] * len(hits)
  for key in x:
    for i, pos in enumerate(hits):
      res[i][key] = tf.convert_to_tensor([x[key][pos]])
  return res

In [23]:
# xs = []
# for i, (x, _) in tqdm(enumerate(batches), total=num_steps, ascii=True, desc='loop'):
# #   xs += get_info(x, 'bfa1A00000934E2403')
#   break

loop:   0%|          | 0/1102904 [00:00<?, ?it/s]


In [26]:
xs = []
for i, (x, _) in tqdm(enumerate(batches), total=num_steps, ascii=True, desc='loop'):
  xs += [x]
  if i == 1000:
    break
#   if i == num_steps:
#     break

loop:   0%|          | 1000/1102904 [00:03<55:39, 329.98it/s] 


In [27]:
x = xs[2]
count = 0
for i in range(1000):
  if xs[i]['duration'].numpy() > 0:
    x = xs[i]
    count += 1
    if count == 9:
      break

In [28]:
def get_doc(xs, docid):
  for x in xs:
    if tf.compat.as_str_any(x['docid'][0].numpy()) == docid:
      return x

In [29]:
model(x)

<tf.Tensor: id=211675, shape=(1, 1), dtype=float32, numpy=array([[0.0213]], dtype=float32)>

In [32]:
x['id']

<tf.Tensor: id=148757, shape=(1,), dtype=string, numpy=array([b'7790864571047670299\t20719m0K1NMLzv'], dtype=object)>

In [33]:
x['duration']

<tf.Tensor: id=148754, shape=(1, 1), dtype=int64, numpy=array([[183]])>

In [30]:
model.layers

In [31]:
model.deep.kw_emb.weights

[<tf.Variable 'wide_deep/deep/kw_emb/embeddings:0' shape=(1000001, 32) dtype=float32, numpy=
 array([[ 0.0341,  0.0431, -0.0375, ...,  0.0173, -0.0411,  0.0428],
        [ 0.018 ,  0.0162,  0.0014, ..., -0.0406, -0.0291, -0.0057],
        [-0.0013, -0.0383, -0.0357, ..., -0.0465, -0.0369, -0.0283],
        ...,
        [ 0.0066, -0.0494, -0.0094, ...,  0.0312,  0.0222,  0.0085],
        [-0.036 , -0.0222, -0.0496, ...,  0.0385, -0.0085,  0.0415],
        [-0.0248,  0.0458, -0.0626, ...,  0.0629, -0.0305, -0.0139]],
       dtype=float32)>]

In [40]:
kw_emb = model.deep.kw_emb.weights[0].numpy()

In [45]:
kw_emb.shape

(1000001, 32)

In [50]:
def kw_sim(kw1, kw2, height=1000000):
  h1 = gezi.hash_int64(kw1)
  h2 = gezi.hash_int64(kw2)
  id1 = h1 % height + 1
  id2 = h2 % height + 1
  return gezi.cosine(kw_emb[id1], kw_emb[id2])

In [51]:
kw_sim('老婆', '媳妇')

-0.065345645

In [52]:
kw_sim('老婆', '孩子')

0.17124209

In [53]:
kw_sim('老婆', '丈夫')

-0.13594934

In [54]:
kw_sim('乒乓球', '丈夫')

0.12070185

In [55]:
kw_sim('乒乓球', '足球')

-0.105204456

In [56]:
kw_sim('篮球', '足球')

0.023281164

In [57]:
kw_sim('篮球', '笔记本')

0.0534849

In [58]:
kw_sim('手机', '笔记本')

0.15951999

In [59]:
kw_sim('苹果', '梨')

0.2748438

In [60]:
kw_sim('苹果', 'mac')

0.07417711

In [61]:
kw_sim('电脑', 'mac')

-0.12172077

In [62]:
kw_sim('电脑', '鼠标')

0.15630038

In [63]:
kw_sim('搜狗', '百度')

-0.13156724

In [64]:
kw_sim('腾讯', '百度')

-0.2364463

In [65]:
kw_sim('美女', '明显')

0.14354429

In [66]:
kw_sim('美女', '明星')

0.33288077

In [67]:
kw_sim('美女', '两性')

0.091602005

In [68]:
kw_sim('美女', '军事')

-0.44393417

In [69]:
kw_sim('时政', '军事')

-0.026391614

In [70]:
kw_sim('政治', '军事')

0.29270053

In [71]:
kw_sim('体育', '足球')

0.30990842

In [72]:
kw_sim('体育', '篮球')

-0.0284415

In [73]:
kw_sim('体育', '乒乓球')

0.19276936

In [74]:
kw_sim('羽毛球', '乒乓球')

-0.045936987

In [75]:
kw_sim('体育', '健身')

0.18916212

In [76]:
kw_sim('体育', '姚明')

0.09824768

In [77]:
kw_sim('体育', '李克强')

-0.32485664

In [78]:
kw_sim('翼装飞行', '翼装')

-0.078976475

In [79]:
kw_sim('肯豆', '时装')

-0.12455144

In [80]:
kw_sim('肯豆', '明星')

-0.1029112

In [81]:
kw_sim('章子怡', '明星')

0.19523934

In [82]:
kw_sim('越南', '新加坡')

0.19065455

In [83]:
kw_sim('越南', '俄罗斯')

0.31473112

In [84]:
kw_sim('越南', '特朗普')

0.015419804

In [85]:
kw_sim('新冠', '特朗普')

0.2662671

In [86]:
kw_sim('美国', '特朗普')

0.14773594

In [87]:
kw_sim('中国', '特朗普')

0.21929994

In [88]:
kw_sim('牛肉', '特朗普')

-0.15745132

In [89]:
kw_sim('牛肉', '美食')

0.36206806

In [39]:
model.deep.user_emb.weights[0].numpy().mean()

0.0014265358

In [36]:
model.deep.doc_emb.weights[0].numpy().mean()

0.00029412145

In [14]:
# 点击塔 wide 的权重0.0054 deep权重 0.0344 wide的影响极小 后面也证明这一点
model.layers[-2].weights

[<tf.Variable 'wide_deep/dense_12/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 0.003 ],
        [-0.0432]], dtype=float32)>,
 <tf.Variable 'wide_deep/dense_12/bias:0' shape=(1,) dtype=float32, numpy=array([-1.2739], dtype=float32)>]

In [12]:
# 这两个doc 第一个位于位置1 未点击            《楞严经》未得谓得未证言证 
# 第二个位于位置11 点击  时长276              不管你相不相信转世和业，修持金刚萨埵法门非常重要
x1 = get_doc(xs, '2050880x18kxgs')
x2 = get_doc(xs, '20508l0l1Sy3rK')

In [14]:
x1['history']

<tf.Tensor: id=164808, shape=(1, 100), dtype=int64, numpy=
array([[ 2497018187339628289,  7454267765482564390,  5249870935199499468,
        -7445710862883341190,  9200600509661742591,  4613952955898248268,
         4613952955898248268,   789118711552849257,  6676206831695143456,
        -4690589300209911868,  5832513682969836603,  5814871040139463220,
         8231775787324756526,  6542662098019923192, -8085199320759549422,
         1624967564591613045,  7981911317382662345,  5062345579739016777,
         7153798303226364985,  8309539223362353568,  1413542521776041921,
        -7045646830384326602,  1173712025618709632,  2457001617728112838,
         -408629747063015612,  5155997678851131417, -1802263287388093764,
         -165857663623427133,   -50534922134768071,   457426426774322503,
         5434037021565601746,  4803698468749676103,  7070228455537301648,
         8985521105019028127, -8766603623861055052, -3076034891908382671,
         6818765462259455698,  6479711340744711493,  

In [14]:
model(x1)

<tf.Tensor: id=244287, shape=(1, 1), dtype=float32, numpy=array([[0.0881]], dtype=float32)>

In [15]:
model.d

<tf.Tensor: id=244049, shape=(1, 1), dtype=float32, numpy=array([[50.2766]], dtype=float32)>

In [16]:
model.w

<tf.Tensor: id=243464, shape=(1, 1), dtype=float32, numpy=array([[-0.348]], dtype=float32)>

In [17]:
model.prob_click

<tf.Tensor: id=244228, shape=(1, 1), dtype=float32, numpy=array([[0.6016]], dtype=float32)>

In [18]:
model.y_click

<tf.Tensor: id=244119, shape=(1, 1), dtype=float32, numpy=array([[0.4122]], dtype=float32)>

In [19]:
gezi.sigmoid(0.0054 * 1.0864 + 0.0344 * (-0.8849) + (-1.3172))

0.20721847628049556

In [20]:
x_all1 = model.deep.x_all

In [21]:
x_all1

<tf.Tensor: id=243834, shape=(1, 94, 32), dtype=float32, numpy=
array([[[ 5.8152e-01,  5.1480e-01,  1.7443e-01, ..., -2.2064e-01,
          8.7434e-01, -3.2459e-01],
        [-9.6461e-03, -2.5568e-02, -5.2258e-03, ..., -6.2161e-04,
          4.1080e-02,  5.5956e-03],
        [-2.8259e+00,  1.9503e+00,  1.9650e+00, ...,  3.3509e-01,
         -1.5168e+00,  5.3758e-03],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]]], dtype=float32)>

In [22]:
model(x2)

<tf.Tensor: id=244663, shape=(1, 1), dtype=float32, numpy=array([[0.0057]], dtype=float32)>

In [23]:
model.d

<tf.Tensor: id=244562, shape=(1, 1), dtype=float32, numpy=array([[-40.3782]], dtype=float32)>

In [24]:
model.w

<tf.Tensor: id=244344, shape=(1, 1), dtype=float32, numpy=array([[-0.725]], dtype=float32)>

In [25]:
model.prob_click

<tf.Tensor: id=244604, shape=(1, 1), dtype=float32, numpy=array([[0.0623]], dtype=float32)>

In [26]:
x_all2 = model.deep.x_all

In [27]:
x_all2

<tf.Tensor: id=244527, shape=(1, 94, 32), dtype=float32, numpy=
array([[[ 0.5815,  0.5148,  0.1744, ..., -0.2206,  0.8743, -0.3246],
        [ 0.04  ,  0.0771, -0.0435, ..., -0.1052, -0.1644, -0.0134],
        [-2.098 ,  1.3853,  1.2746, ...,  0.4001, -0.9486, -0.1274],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)>

In [28]:
x1['x_all'] = x_all2
model(x1)
# 可以看到wide部分几乎没有影响到最终得分

<tf.Tensor: id=244837, shape=(1, 1), dtype=float32, numpy=array([[0.0042]], dtype=float32)>

In [29]:
model.d

<tf.Tensor: id=244737, shape=(1, 1), dtype=float32, numpy=array([[-40.3782]], dtype=float32)>

In [30]:
model.w

<tf.Tensor: id=244702, shape=(1, 1), dtype=float32, numpy=array([[-0.348]], dtype=float32)>

In [31]:
model.prob_click

<tf.Tensor: id=244778, shape=(1, 1), dtype=float32, numpy=array([[0.0624]], dtype=float32)>

In [32]:
# 保持wide不变 更好 deep  的fields x_all 点击改了只是稍稍高了一点点 几乎可以忽略
gezi.sigmoid(0.0054 * 1.0864 + 0.0344 * (-50.9848) + (-1.3172))

0.04456531408785941

In [33]:
def rename(field):
  for x in maps:
#     print(x, len(x))
#     print('0', field, x[0], x[1])
    field = field.replace(x[0], x[1])
#     print('1', field, x[0], x[1])
  return field

In [34]:
other_fields = [x.name.split('/')[-2] for x in model.variables if x.name.endswith('embeddings:0')and not '/emb/emb/' in x.name]
other_fields[model.deep.history_index:model.deep.history_index] = model.deep.history_fields
onehot_fields = gezi.get_global('fields')
maps = [x.strip().split() for x in open('../conf/fields_map.txt')]
onehot_fields2 = [rename(x) for x in onehot_fields]
fields = other_fields + onehot_fields2
fields2 = other_fields + onehot_fields
# [(x, y) for x, y in zip(onehot_fields, onehot_fields2)]

In [35]:
import copy
x_all2_npy = x_all2.numpy()
x_all1_npy = x_all1.numpy()

In [36]:
x_all2_npy

array([[[ 0.5815,  0.5148,  0.1744, ..., -0.2206,  0.8743, -0.3246],
        [ 0.04  ,  0.0771, -0.0435, ..., -0.1052, -0.1644, -0.0134],
        [-2.098 ,  1.3853,  1.2746, ...,  0.4001, -0.9486, -0.1274],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)

In [37]:
x_all = x_all2_npy.copy()
x_all[0][1] = x_all1_npy[0][1]
x_all

array([[[ 5.8152e-01,  5.1480e-01,  1.7443e-01, ..., -2.2064e-01,
          8.7434e-01, -3.2459e-01],
        [-9.6461e-03, -2.5568e-02, -5.2258e-03, ..., -6.2161e-04,
          4.1080e-02,  5.5956e-03],
        [-2.0980e+00,  1.3853e+00,  1.2746e+00, ...,  4.0013e-01,
         -9.4864e-01, -1.2743e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]]], dtype=float32)

In [38]:
x2['x_all'] = x_all
model(x2)
model.prob_click

<tf.Tensor: id=244954, shape=(1, 1), dtype=float32, numpy=array([[0.0795]], dtype=float32)>

In [39]:
l = []
for i in tqdm(range(len(fields))):
  x_all = x_all1_npy.copy()
  x_all[0][i] = x_all2_npy[0][i]
  x1['x_all'] = x_all
  model(x1)
  prob_click1 = model.prob_click.numpy()[0][0]
  x_all = x_all2_npy.copy()
  x_all[0][i] = x_all1_npy[0][i]
  x2['x_all'] = x_all
  model(x2)
  prob_click2 = model.prob_click.numpy()[0][0]
  l.append((i, fields[i], fields2[i], prob_click1, prob_click2, prob_click2 - prob_click1))
  l.sort(key=lambda x: x[-1], reverse=True)
gezi.pprint(pd.DataFrame(l, columns=['fid', 'field', 'ori_field', 'pclick1', 'pclick2', 'delta']))

100%|██████████| 94/94 [00:02<00:00, 37.13it/s]


 +-------+------------------------------+-----------------+-----------+-----------+-----------+
|   fid | field                        | ori_field       |   pclick1 |   pclick2 |     delta |
|-------+------------------------------+-----------------+-----------+-----------+-----------|
|    29 | IArtKw2                      | IATKWSE         |  0.415008 | 0.0965749 | -0.318433 |
|    26 | IAccountId                   | IACID           |  0.463604 | 0.135509  | -0.328095 |
|    65 | MInterestArtPornScoreTag     | MITATPORNSTG    |  0.525344 | 0.103286  | -0.422058 |
|    63 | MInterestArtPornScoreAccount | MITATPORNSAC    |  0.551189 | 0.0938884 | -0.457301 |
|    33 | IArtTopic                    | IATTP           |  0.542161 | 0.0666729 | -0.475488 |
|    80 | XArtShowClickCnt             | XATSWCL         |  0.561255 | 0.0741115 | -0.487144 |
|     2 | history                      | history         |  0.556743 | 0.062308  | -0.494435 |
|    64 | MInterestArtPornScoreKw      | MITATPO

In [51]:
x_all = x_all2_npy.copy()
x_all[0][26] = x_all1_npy[0][26]
x_all[0][29] = x_all1_npy[0][29]
x_all[0][65] = x_all1_npy[0][65]
x_all[0][63] = x_all1_npy[0][63]
x_all[0][33] = x_all1_npy[0][33]
x2['x_all'] = x_all
model(x2)
model.prob_click

<tf.Tensor: id=278983, shape=(1, 1), dtype=float32, numpy=array([[0.4575]], dtype=float32)>

In [40]:
model

In [ ]:
model.logit

In [ ]:
model.prob

In [ ]:
model.prob_click

In [ ]:
model.prob_dur

In [ ]:
x['ol_pred']

In [ ]:
x['ol_pred_click']

In [ ]:
x['ol_pred_dur']

In [ ]:
x['ol_pred_click'] ** 1.3 * x['ol_pred_dur'] ** 0.7

In [ ]:
model.prob_click ** 1.3 * model.prob_dur ** 0.7

In [ ]:
x['ori_lr_score']

In [ ]:
model.deep.before_mlp

In [ ]:
dim = tf.cast(tf.cast(tf.shape(model.deep.before_mlp)[1], tf.float32) ** 0.5, tf.int32)
dim

In [ ]:
intersection = tf.reshape(model.deep.before_mlp, [dim, dim]).numpy()

In [ ]:
intersection[0][1]

In [ ]:
for i in range(dim):
  intersection[i][i] = 0.

In [ ]:
intersection.shape

In [ ]:
# intersection = tf.minimum(intersection, 5)

In [ ]:
tf.reduce_sum(tf.cast(intersection > 2, tf.int32))

In [ ]:
X = list(range(dim))
X = fields
heat = go.Heatmap(z=intersection,
                  x=X,
                  y=X,
                  xgap=1, ygap=1,
                  )
layout = go.Layout(
width=2000, height=2000,
xaxis_showgrid=False,
yaxis_showgrid=False,
yaxis_autorange='reversed'
)
fig=go.Figure(data=[heat], layout=layout)
py.iplot(fig)

In [ ]:
d = pd.DataFrame(intersection)
d

In [ ]:
x['history'].shape

In [ ]:
l = [(d[i] ** 2).mean() for i in range(len(d))]
l2 = [d[i].mean() for i in range(len(d))]

In [ ]:
[(i, fields[i]) for i in range(len(d)) if l[i] == 0]

In [ ]:
sorted([(l[i], fields[i]) for i in range(len(d))], reverse=True)

In [ ]:
sorted([(l2[i], fields[i]) for i in range(len(d))], reverse=True)

In [ ]:
m = []
m2 = []
for i in range(len(d)):
  for j in range(len(d)):
    if j > i:
      m.append((d[i][j], fields[i], fields[j]))
      m2.append((d[i][j] ** 2, d[i][j], fields[i], fields[j]))

In [ ]:
sorted(m, reverse=True)[:100]

In [ ]:
sorted(m, reverse=False)[:100]

In [ ]:
sorted(m2, reverse=True)[:100]

In [ ]:
x['history'].shape

In [ ]:
index = [u.numpy().decode('gbk').split('\a', 1) for u in info[x['id'][0].numpy()]['index'] if u.numpy().decode('gbk') != '\x01']
[(rename(field), val) for field, val in index]

In [ ]:
len(x['value'][0])

In [ ]:
len([(rename(field), val) for field, val in index if not 'last' in field and not 'long' in field])

In [ ]:
x['mid']

In [ ]:
len([(rename(field), val) for field, val in index if 'last' in field])

In [ ]:
len([(rename(field), val) for field, val in index if 'long' in field])

In [ ]:
from projects.feed.rank.utils.doc_info.KV import KV
kv = KV()
titles = kv.get_titles([u.decode() for u in info[x['id'][0].numpy()]['history'].numpy() if u.decode() not in ['\x01', 'null']])
titles

In [ ]:
kv.get_titles([x['docid'][0].numpy().decode()])

In [ ]:
docs = [u.decode() for u in info[x['id'][0].numpy()]['history'].numpy() if u.decode() != '\x01']
docs

In [ ]:
keys = list(info.keys())
index = [u.numpy().decode('gbk').split('\a', 1) for u in info[keys[0]]['index'] if u.numpy().decode('gbk') != '\x01']
[(rename(field), val) for field, val in index]

In [ ]:
info[keys[0]]

In [ ]:
doc_embs = model.deep.doc_emb(tf.constant([gezi.hash_int64(u) for u in docs]))

In [ ]:
doc2doc = tf.matmul(doc_embs, tf.transpose(doc_embs))

In [ ]:
X = [titles.get(u, 'null') for u in docs]
heat = go.Heatmap(z=doc2doc,
                  x=X,
                  y=X,
                  xgap=1, ygap=1,
                  )
layout = go.Layout(
width=1000, height=1000,
xaxis_showgrid=False,
yaxis_showgrid=False,
yaxis_autorange='reversed'
)
fig=go.Figure(data=[heat], layout=layout)
py.iplot(fig)